In [1]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode

from nipype.interfaces.slicer.registration import brainsresample
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.freesurfer.preprocess import MRIConvert, ApplyVolTransform
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.fsl.maths import ApplyMask
from nipype.interfaces.fsl.preprocess import FLIRT, SUSAN
from nipype.interfaces.freesurfer.model import Binarize

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

#other study-specific variables
project_home = '/Volumes/active-19/BABIES/BABIES_perfusion'
raw_dir = project_home + '/Raw'
subjects_list = open(project_home + '/Misc/Subjects.txt').read().splitlines()
#subjects_list = ['012']
output_dir = project_home + '/Proc'
wkflow_dir = project_home + '/Workflows'
template = project_home + '/Templates/T2wtemplate_2mm.nii'

#Population specific variables for ASL
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header at position 0043,107f (corresponds to #coils?)
postlabel_delay = 1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
T1_tissue = 1.2 #estimated T1 of grey matter in seconds
TR = 4.844 #repetition time

smoothing_kernel = 4 # in mm

In [2]:
## File handling nodes

# Select subjects
infosource = Node(IdentityInterface(fields=['subjid', 'volume']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]


# SelectFiles
templates = {'pw_volume': raw_dir + '/{subjid}-C-T1/pw.nii',
             'anat_volume': raw_dir + '/{subjid}-C-T1/processed_t2.nii',
             'pd_volume': raw_dir + '/{subjid}-C-T1/pd.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')


# Datasink
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
# DataSink output substitutions (for ease of folder naming)
substitutions = [('_subjid_', ''),
                ('volume_',''),
                ('_reoriented',''),
                ('_warped','')]
datasink.inputs.substitutions = substitutions

In [3]:
## File Processing nodes

# convert files to nifti
mri_convert = Node(MRIConvert(out_type='nii',
                                conform_size=2,
                                crop_size= (128, 128, 128)), 
                   name='mri_convert')

# reorient data for consistency
reorient_anat = Node(Reorient2Std(output_type='NIFTI'),
                     name='reorient_anat')

# reorient data for consistency
reorient_pd = Node(Reorient2Std(output_type='NIFTI'),
                   name='reorient_pd')

# reorient data for consistency
reorient_pw = Node(Reorient2Std(output_type='NIFTI'),
                   name='reorient_pw')


# Binarize -  binarize and dilate image to create a brainmask
binarize = Node(Binarize(min=0.5,
                         dilate=2,
                         erode=1,
                         out_type='nii'),
                name='binarize')

    

reg2anat = Node(FLIRT(out_matrix_file = 'xform.mat'), name = 'reg2anat')
applyxform = Node(FLIRT(apply_xfm = True), name = 'applyxform')

# Mask brain in pw and pd volumes
applyMask_pd = Node(ApplyMask(output_type='NIFTI'), 
                    name='applyMask_pd')

applyMask_pw = Node(ApplyMask(output_type='NIFTI'), 
                    name='applyMask_pw')

# N3 bias correction using MINC tools, will be necessary for babies
# nu_correct = Node(fs.MNIBiasCorrection(), name='nu_correct')

In [4]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = os.path.abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))

In [5]:
# Create a flow for preprocessing anat + asl volumes 
preprocflow = Workflow(name='preprocflow')

# Connect all components of the preprocessing workflow
preprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, mri_convert, [('anat_volume', 'in_file')]),
                     (mri_convert, reorient_anat, [('out_file', 'in_file')]),    
                     (selectfiles, reorient_pw, [('pw_volume', 'in_file')]), 
                     (selectfiles, reorient_pd, [('pd_volume', 'in_file')]),
                     (reorient_anat, reg2anat, [('out_file', 'reference')]),
                     (reorient_pw, reg2anat, [('out_file', 'in_file')]),
                     (reg2anat, applyxform, [('out_matrix_file','in_matrix_file')]),
                     (reorient_anat, applyxform, [('out_file','reference')]),
                     (reorient_pd, applyxform, [('out_file','in_file')]),
                     (reorient_anat, binarize, [('out_file','in_file')]),
                     (applyxform, applyMask_pd, [('out_file','in_file')]),
                     (binarize, applyMask_pd, [('binary_file','mask_file')]),
                     (reg2anat, applyMask_pw, [('out_file','in_file')]),
                     (binarize, applyMask_pw, [('binary_file','mask_file')]),
                     (reg2anat, make_coreg_img, [('out_file','epi')]),
                     (reorient_anat, make_coreg_img, [('out_file','anat')]),
                     (make_coreg_img, datasink, [('coreg_file','coreg_check')]),
                     (reg2anat, make_checkmask_img, [('out_file','epi')]),
                     (binarize, make_checkmask_img, [('binary_file','brainmask')]),
                     (make_checkmask_img, datasink, [('maskcheck_file','masked_check')]),
                     (applyMask_pd, datasink, [('out_file','masked_pd')]),
                     (applyMask_pw, datasink, [('out_file','masked_pw')]),
                     (reorient_anat, datasink, [('out_file', 'reorient_anat')])
                    ])
preprocflow.base_dir = wkflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 2})


171109-14:20:42,630 workflow INFO:
	 Generated workflow graph: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/graph.dot.png (graph2use=flat, simple_form=True).
171109-14:20:42,857 workflow INFO:
	 Workflow preprocflow settings: ['check', 'execution', 'logging']
171109-14:20:43,160 workflow INFO:
	 Running in parallel.
171109-14:20:43,172 workflow INFO:
	 Executing: selectfiles.a19 ID: 0
171109-14:20:43,206 workflow INFO:
	 Executing: selectfiles.a18 ID: 13
171109-14:20:43,209 workflow INFO:
	 Executing node selectfiles.a19 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_077x/selectfiles
171109-14:20:43,231 workflow INFO:
	 Executing node selectfiles.a18 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_072/selectfiles
171109-14:20:43,739 workflow INFO:
	 [Job finished] jobname: selectfiles.a19 jobid: 0
171109-14:20:43,745 workflow INFO:
	 Executing: mri_convert.a19 ID: 1
171109-14:20:43,761 workflow INFO:

171109-14:20:51,690 workflow INFO:
	 [Job finished] jobname: reorient_pw.a17 jobid: 30
171109-14:20:51,695 workflow INFO:
	 Executing: reg2anat.a17 ID: 31
171109-14:20:51,714 workflow INFO:
	 [Job finished] jobname: reg2anat.a17 jobid: 31
171109-14:20:51,716 workflow INFO:
	 Executing: reorient_pd.a17 ID: 35
171109-14:20:51,731 workflow INFO:
	 [Job finished] jobname: reorient_pd.a17 jobid: 35
171109-14:20:51,737 workflow INFO:
	 Executing: applyMask_pw.a17 ID: 32
171109-14:20:51,757 workflow INFO:
	 [Job finished] jobname: applyMask_pw.a17 jobid: 32
171109-14:20:51,759 workflow INFO:
	 Executing: make_coreg_img.a17 ID: 33
171109-14:20:51,777 workflow INFO:
	 [Job finished] jobname: make_coreg_img.a17 jobid: 33
171109-14:20:51,782 workflow INFO:
	 Executing: make_checkmask_img.a17 ID: 34
171109-14:20:51,799 workflow INFO:
	 [Job finished] jobname: make_checkmask_img.a17 jobid: 34
171109-14:20:51,801 workflow INFO:
	 Executing: applyxform.a17 ID: 36
171109-14:20:51,824 workflow INFO:
	 

171109-14:21:00,527 workflow INFO:
	 Executing: selectfiles.a14 ID: 65
171109-14:21:00,574 workflow INFO:
	 Executing node selectfiles.a14 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_087/selectfiles
171109-14:21:00,891 workflow INFO:
	 [Job finished] jobname: selectfiles.a14 jobid: 65
171109-14:21:00,896 workflow INFO:
	 Executing: datasink.a15 ID: 64
171109-14:21:00,973 workflow INFO:
	 Executing: mri_convert.a14 ID: 66
171109-14:21:00,975 workflow INFO:
	 Executing node datasink.a15 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_076x/datasink
171109-14:21:00,987 workflow INFO:
	 [Job finished] jobname: mri_convert.a14 jobid: 66
171109-14:21:01,204 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/reorient_anat/_subjid_076x/processed_t2_out_reoriented.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/reorient_anat/076x/processed_t2_out.nii
171109-14:21:01,578 interface INFO:
	 sub: /Volume

	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pd/_subjid_064x/pd_reoriented_flirt_masked.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pd/064x/pd_flirt_masked.nii
171109-14:21:11,574 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pw/_subjid_064x/pw_reoriented_flirt_masked.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pw/064x/pw_flirt_masked.nii
171109-14:21:11,587 workflow INFO:
	 Executing: reorient_anat.a12 ID: 93
171109-14:21:11,622 workflow INFO:
	 [Job finished] jobname: reorient_anat.a12 jobid: 93
171109-14:21:12,957 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/coreg_check/_subjid_064x/coregistration.png -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/coreg_check/064x/coregistration.png
171109-14:21:13,6 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_check/_subjid_064x/maskcheck.png -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_che

171109-14:21:21,920 workflow INFO:
	 Executing: binarize.a10 ID: 120
171109-14:21:21,937 workflow INFO:
	 [Job finished] jobname: binarize.a10 jobid: 120
171109-14:21:21,940 workflow INFO:
	 Executing: reorient_pw.a10 ID: 121
171109-14:21:21,954 workflow INFO:
	 [Job finished] jobname: reorient_pw.a10 jobid: 121
171109-14:21:21,959 workflow INFO:
	 Executing: reg2anat.a10 ID: 122
171109-14:21:21,978 workflow INFO:
	 [Job finished] jobname: reg2anat.a10 jobid: 122
171109-14:21:21,980 workflow INFO:
	 Executing: reorient_pd.a10 ID: 126
171109-14:21:21,994 workflow INFO:
	 [Job finished] jobname: reorient_pd.a10 jobid: 126
171109-14:21:22,0 workflow INFO:
	 Executing: applyMask_pw.a10 ID: 123
171109-14:21:22,20 workflow INFO:
	 [Job finished] jobname: applyMask_pw.a10 jobid: 123
171109-14:21:22,23 workflow INFO:
	 Executing: make_coreg_img.a10 ID: 124
171109-14:21:22,40 workflow INFO:
	 [Job finished] jobname: make_coreg_img.a10 jobid: 124
171109-14:21:22,45 workflow INFO:
	 Executing: ma

171109-14:21:30,489 workflow INFO:
	 [Job finished] jobname: applyxform.a08 jobid: 153
171109-14:21:30,494 workflow INFO:
	 Executing: applyMask_pd.a08 ID: 154
171109-14:21:30,525 workflow INFO:
	 [Job finished] jobname: applyMask_pd.a08 jobid: 154
171109-14:21:30,527 workflow INFO:
	 Executing: selectfiles.a07 ID: 156
171109-14:21:30,569 workflow INFO:
	 Executing node selectfiles.a07 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_035/selectfiles
171109-14:21:30,868 workflow INFO:
	 [Job finished] jobname: selectfiles.a07 jobid: 156
171109-14:21:30,874 workflow INFO:
	 Executing: datasink.a08 ID: 155
171109-14:21:30,940 workflow INFO:
	 Executing: mri_convert.a07 ID: 157
171109-14:21:30,942 workflow INFO:
	 Executing node datasink.a08 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_036/datasink
171109-14:21:30,954 workflow INFO:
	 [Job finished] jobname: mri_convert.a07 jobid: 157
171109-14:21:31,143 interface INFO:
	 

	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/reorient_anat/_subjid_032/processed_t2_out_reoriented.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/reorient_anat/032/processed_t2_out.nii
171109-14:21:39,985 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pd/_subjid_032/pd_reoriented_flirt_masked.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pd/032/pd_flirt_masked.nii
171109-14:21:41,282 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pw/_subjid_032/pw_reoriented_flirt_masked.nii -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_pw/032/pw_flirt_masked.nii
171109-14:21:41,394 workflow INFO:
	 Executing: reorient_anat.a05 ID: 184
171109-14:21:41,437 workflow INFO:
	 [Job finished] jobname: reorient_anat.a05 jobid: 184
171109-14:21:42,572 interface INFO:
	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/coreg_check/_subjid_032/coregistration.png -> /Volumes/active-19/BABIES/BABIES_perfu

	 sub: /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_check/_subjid_027/maskcheck.png -> /Volumes/active-19/BABIES/BABIES_perfusion/Proc/masked_check/027/maskcheck.png
171109-14:21:50,830 workflow INFO:
	 [Job finished] jobname: datasink.a04 jobid: 207
171109-14:21:50,835 workflow INFO:
	 Executing: binarize.a03 ID: 211
171109-14:21:50,852 workflow INFO:
	 [Job finished] jobname: binarize.a03 jobid: 211
171109-14:21:50,854 workflow INFO:
	 Executing: reorient_pw.a03 ID: 212
171109-14:21:50,877 workflow INFO:
	 [Job finished] jobname: reorient_pw.a03 jobid: 212
171109-14:21:50,881 workflow INFO:
	 Executing: reg2anat.a03 ID: 213
171109-14:21:50,901 workflow INFO:
	 [Job finished] jobname: reg2anat.a03 jobid: 213
171109-14:21:50,903 workflow INFO:
	 Executing: reorient_pd.a03 ID: 217
171109-14:21:50,915 workflow INFO:
	 [Job finished] jobname: reorient_pd.a03 jobid: 217
171109-14:21:50,920 workflow INFO:
	 Executing: applyMask_pw.a03 ID: 214
171109-14:21:50,941 workflow INFO:
	 [

171109-14:21:59,594 workflow INFO:
	 [Job finished] jobname: make_coreg_img.a01 jobid: 241
171109-14:21:59,599 workflow INFO:
	 Executing: make_checkmask_img.a01 ID: 242
171109-14:21:59,617 workflow INFO:
	 [Job finished] jobname: make_checkmask_img.a01 jobid: 242
171109-14:21:59,619 workflow INFO:
	 Executing: applyxform.a01 ID: 244
171109-14:21:59,641 workflow INFO:
	 [Job finished] jobname: applyxform.a01 jobid: 244
171109-14:21:59,646 workflow INFO:
	 Executing: applyMask_pd.a01 ID: 245
171109-14:21:59,664 workflow INFO:
	 [Job finished] jobname: applyMask_pd.a01 jobid: 245
171109-14:21:59,666 workflow INFO:
	 Executing: selectfiles.a00 ID: 247
171109-14:21:59,702 workflow INFO:
	 Executing node selectfiles.a00 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/preprocflow/_subjid_012/selectfiles
171109-14:22:00,17 workflow INFO:
	 [Job finished] jobname: selectfiles.a00 jobid: 247
171109-14:22:00,22 workflow INFO:
	 Executing: datasink.a01 ID: 246
171109-14:22:00,67 work

In [6]:
## File handling nodes for CBF proc

# Select subjects
cbfinfosource = Node(IdentityInterface(fields=['subjid']),
                  name='cbfinfosource')
cbfinfosource.iterables = [('subjid', subjects_list)]


# SelectFiles
templates = {'pw_volume': output_dir + '/masked_pw/{subjid}/pw_flirt_masked.nii',
            'pd_volume': output_dir + '/masked_pd/{subjid}/pd_flirt_masked.nii',
            'anat_volume': output_dir + '/reorient_anat/{subjid}/processed_t2_out.nii'}
cbfselectfiles = Node(SelectFiles(templates), name='cbfselectfiles')

# Datasink
cbfdatasink = Node(DataSink(), name='cbfdatasink')
cbfdatasink.inputs.base_directory = output_dir
cbfdatasink.inputs.container = output_dir
# DataSink output substitutions (for ease of folder naming)
substitutions = [('_subjid_', '')]
cbfdatasink.inputs.substitutions = substitutions

In [7]:
## Custom functions

#quantify CBF from PW volume (Alsop MRIM 2015 method)
def quantify_cbf_alsop(pw_volume,pd_volume,sat_time,postlabel_delay,T1_blood,labeling_time,efficiency,partition_coeff,TR,T1_tissue,scaling_factor,nex_asl):
    import os
    import nibabel as nib
    from numpy import exp
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    # set variables
    pw_nifti1 = nib.load(pw_volume)
    pw_data = pw_nifti1.get_data()
    pw_data = pw_data.astype(float)
    pd_nifti1 = nib.load(pd_volume)
    pd_data = pd_nifti1.get_data()
    pd_data = pd_data.astype(float)
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    pd_factor = 1/(1-exp((-1*TR)/T1_tissue))
    
    cbf_numerator = conversion*partition_coeff*pw_data*exp(postlabel_delay/T1_blood)
    cbf_denominator = sat_time*efficiency*T1_blood*scaling_factor*nex_asl*pd_data*pd_factor*(1-exp((-1*labeling_time)/T1_blood))
    cbf_data = cbf_numerator/cbf_denominator
    
    cbf_volume = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_volume, 'alsop_cbf.nii')
    cbf_path = os.path.abspath('alsop_cbf.nii')
    return cbf_path

quant_cbf_alsop = Node(name='quant_cbf_alsop',
                interface=Function(input_names=['pw_volume','pd_volume',
                                                'sat_time','postlabel_delay',
                                                'T1_blood','labeling_time',
                                                'efficiency','partition_coeff',
                                                'TR','T1_tissue','scaling_factor',
                                                'nex_asl'],
                                  output_names=['cbf_volume'],
                                  function=quantify_cbf_alsop))
quant_cbf_alsop.inputs.sat_time=sat_time
quant_cbf_alsop.inputs.postlabel_delay=postlabel_delay
quant_cbf_alsop.inputs.T1_blood=T1_blood
quant_cbf_alsop.inputs.labeling_time=labeling_time
quant_cbf_alsop.inputs.efficiency=efficiency
quant_cbf_alsop.inputs.partition_coeff=partition_coeff
quant_cbf_alsop.inputs.TR=TR
quant_cbf_alsop.inputs.T1_tissue=T1_tissue
quant_cbf_alsop.inputs.scaling_factor=scaling_factor
quant_cbf_alsop.inputs.nex_asl=nex_asl

# Calculate Brightness Threshold for SUSAN

# Brightness threshold should be 0.75 * the contrast between the median brain intensity and the background
def calc_brightness_threshold(func_vol):
    import nibabel as nib
    from numpy import median, where
    
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    func_nifti1 = nib.load(func)
    func_data = func_nifti1.get_data()
    func_data = func_data.astype(float)
    
    brain_values = where(func_data > 0)
    median_thresh = median(brain_values)
    brightness_threshold = 0.75 * median_thresh
    return(brightness_threshold)

In [8]:
## Normalizing data for first and second level analysis

# Calculate brightness threshold
calc_bright_thresh = Node(Function(input_names=['func_vol'],
                                   output_names=['brightness_threshold'],
                                   function=calc_brightness_threshold), 
                          name='calc_bright_thresh')

# Smooth parameter estimates- input brightness_threshold and in_file; output smoothed_file
smooth = Node(SUSAN(fwhm=smoothing_kernel), 
              name='smooth')

# Register subject's anatomy to the template
linearReg = Node(FLIRT(output_type='NIFTI',
                       reference=template),
                 name='linearReg')

## Register CBF vol to MNI space
# Volume Transformation - transform the cbf volume into MNI space
warpCBF = Node(ApplyVolTransform(inverse=False,
                                 target_file=template),
               name='warpCBF')

In [9]:
# create a flow for quantifying CBF and warping to MNI space.
cbfprocflow = Workflow(name='cbfprocflow')

# connect the nodes
cbfprocflow.connect([(cbfinfosource, cbfselectfiles, [('subjid', 'subjid')]),
                     (cbfselectfiles, quant_cbf_alsop, [('pw_volume', 'pw_volume')]),
                     (cbfselectfiles, quant_cbf_alsop, [('pd_volume', 'pd_volume')]),
                     (quant_cbf_alsop, cbfdatasink, [('cbf_volume', 'alsop_quant_cbf')]),
                     (cbfselectfiles, linearReg, [('anat_volume', 'in_file')]),
                     (linearReg, cbfdatasink, [('out_file','linwarped_anat')]),
                     (linearReg, warpCBF, [('out_matrix_file', 'fsl_reg_file')]),
                     (quant_cbf_alsop, calc_bright_thresh, [('cbf_volume','func_vol')]),
                     (calc_bright_thresh, smooth, [('brightness_threshold','brightness_threshold')]),
                     (quant_cbf_alsop, smooth, [('cbf_volume','in_file')]),
                     (smooth, warpCBF, [('smoothed_file', 'source_file')]),
                     (warpCBF, cbfdatasink, [('transformed_file', 'warped_cbf_vol')])
                    ]),
cbfprocflow.base_dir = wkflow_dir
cbfprocflow.write_graph(graph2use='flat')
cbfprocflow.run('MultiProc', plugin_args={'n_procs': 2})

171109-14:22:08,111 workflow INFO:
	 Generated workflow graph: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/graph.dot.png (graph2use=flat, simple_form=True).
171109-14:22:08,134 workflow INFO:
	 Workflow cbfprocflow settings: ['check', 'execution', 'logging']
171109-14:22:08,337 workflow INFO:
	 Running in parallel.
171109-14:22:08,345 workflow INFO:
	 Executing: cbfselectfiles.a19 ID: 0
171109-14:22:08,378 workflow INFO:
	 Executing: cbfselectfiles.a18 ID: 7
171109-14:22:08,380 workflow INFO:
	 Executing node cbfselectfiles.a19 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_077x/cbfselectfiles
171109-14:22:08,400 workflow INFO:
	 Executing node cbfselectfiles.a18 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_072/cbfselectfiles
171109-14:22:08,993 workflow INFO:
	 [Job finished] jobname: cbfselectfiles.a19 jobid: 0
171109-14:22:08,999 workflow INFO:
	 Executing: quant_cbf_alsop.a19 ID: 1
171109-14:

171109-14:22:09,737 workflow INFO:
	 Executing: cbfselectfiles.a16 ID: 21
171109-14:22:09,750 workflow DEBUG:
	 Executing node
171109-14:22:09,753 workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
171109-14:22:09,757 workflow INFO:
	 Executing node cbfselectfiles.a16 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_062/cbfselectfiles
171109-14:22:09,763 workflow DEBUG:
	 Output dir: ['_node.pklz', '_inputs.pklz', '_report', 'result_cbfselectfiles.pklz', '_0x8f8a005935cfda4de24c3db58693cd13.json']
171109-14:22:09,771 workflow DEBUG:
	 Output dir: ['_node.pklz', '_inputs.pklz', '_report', 'result_cbfselectfiles.pklz', '_0x8f8a005935cfda4de24c3db58693cd13.json']
171109-14:22:09,778 workflow DEBUG:
	 Found hashfiles: ['/Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_062/cbfselectfiles/_0x8f8a005935cfda4de24c3db58693cd13.json']
171109-14:22:09,781 workflow DEBUG:
	 Final hashfile: /Volumes/active-19/BABIES/BABIES_p

171109-14:22:10,444 workflow INFO:
	 Executing: quant_cbf_alsop.a16 ID: 22
171109-14:22:10,462 workflow INFO:
	 [Job finished] jobname: quant_cbf_alsop.a16 jobid: 22
171109-14:22:10,464 workflow INFO:
	 Executing: linearReg.a16 ID: 25
171109-14:22:10,477 workflow INFO:
	 [Job finished] jobname: linearReg.a16 jobid: 25
171109-14:22:10,481 workflow INFO:
	 Executing: calc_bright_thresh.a16 ID: 23
171109-14:22:10,496 workflow INFO:
	 Executing: cbfselectfiles.a15 ID: 28
171109-14:22:10,497 workflow INFO:
	 Executing node calc_bright_thresh.a16 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_062/calc_bright_thresh
171109-14:22:10,504 workflow DEBUG:
	 Output dir: ['_report', 'result_calc_bright_thresh.pklz', '_node.pklz', '_inputs.pklz']
171109-14:22:10,510 workflow DEBUG:
	 Output dir: ['_report', 'result_calc_bright_thresh.pklz', '_node.pklz', '_inputs.pklz']171109-14:22:10,510 workflow INFO:
	 Executing node cbfselectfiles.a15 in dir: /Volumes/active-19/

	 Found hashfiles: ['/Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_087/cbfselectfiles/_0x7895a35dad27abc7577897f1859fa0df.json']
171109-14:22:10,854 workflow DEBUG:
	 Final hashfile: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_087/cbfselectfiles/_0x7895a35dad27abc7577897f1859fa0df.json
171109-14:22:10,858 workflow DEBUG:
	 updatehash=False, overwrite=None, always_run=True, hash_exists=True
171109-14:22:10,862 workflow DEBUG:
	 Node hash: 7895a35dad27abc7577897f1859fa0df171109-14:22:10,862 workflow DEBUG:
	 saved results in /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_076x/cbfselectfiles/result_cbfselectfiles.pklz

171109-14:22:10,872 workflow DEBUG:
	 Rerunning node:
updatehash = False, self.overwrite = None, self._interface.always_run = True, os.path.exists(/Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_087/cbfselectfiles/_0x7895a35dad27abc7577897f1859fa0df.json) = True, hash_

171109-14:22:11,481 workflow INFO:
	 [Job finished] jobname: quant_cbf_alsop.a14 jobid: 36
171109-14:22:11,483 workflow INFO:
	 Executing: linearReg.a14 ID: 39
171109-14:22:11,497 workflow INFO:
	 [Job finished] jobname: linearReg.a14 jobid: 39
171109-14:22:11,501 workflow INFO:
	 Executing: calc_bright_thresh.a14 ID: 37
171109-14:22:11,516 workflow INFO:
	 Executing: cbfselectfiles.a13 ID: 42
171109-14:22:11,517 workflow INFO:
	 Executing node calc_bright_thresh.a14 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_087/calc_bright_thresh
171109-14:22:11,524 workflow DEBUG:
	 Output dir: ['result_calc_bright_thresh.pklz', '_inputs.pklz', '_node.pklz', '_report']
171109-14:22:11,531 workflow INFO:
	 Executing node cbfselectfiles.a13 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_064x/cbfselectfiles171109-14:22:11,532 workflow DEBUG:
	 Output dir: ['result_calc_bright_thresh.pklz', '_inputs.pklz', '_node.pklz', '_report']



...

171109-14:22:17,816 workflow INFO:
	 [Job finished] jobname: quant_cbf_alsop.a02 jobid: 120
171109-14:22:17,818 workflow INFO:
	 Executing: linearReg.a02 ID: 123
171109-14:22:17,831 workflow INFO:
	 [Job finished] jobname: linearReg.a02 jobid: 123
171109-14:22:17,835 workflow INFO:
	 Executing: calc_bright_thresh.a02 ID: 121
171109-14:22:17,851 workflow INFO:
	 Executing: cbfselectfiles.a01 ID: 126
171109-14:22:17,852 workflow INFO:
	 Executing node calc_bright_thresh.a02 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_021/calc_bright_thresh
171109-14:22:17,867 workflow DEBUG:
	 Output dir: ['_inputs.pklz', 'result_calc_bright_thresh.pklz', '_node.pklz', '_report']
171109-14:22:17,872 workflow INFO:
	 Executing node cbfselectfiles.a01 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_020/cbfselectfiles171109-14:22:17,873 workflow DEBUG:
	 Output dir: ['_inputs.pklz', 'result_calc_bright_thresh.pklz', '_node.pklz', '_report

	 Found hashfiles: ['/Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_012/cbfselectfiles/_0x365af3ccc5d2ae8701a3e06c2aaa8984.json']
171109-14:22:18,229 workflow DEBUG:
	 saved results in /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_020/cbfselectfiles/result_cbfselectfiles.pklz171109-14:22:18,230 workflow DEBUG:
	 Final hashfile: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_012/cbfselectfiles/_0x365af3ccc5d2ae8701a3e06c2aaa8984.json

171109-14:22:18,233 workflow DEBUG:
	 updatehash=False, overwrite=None, always_run=True, hash_exists=True
171109-14:22:18,237 workflow DEBUG:
	 Node hash: 365af3ccc5d2ae8701a3e06c2aaa8984
171109-14:22:18,242 workflow DEBUG:
	 writing post-exec report to /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_020/cbfselectfiles/_report/report.rst
171109-14:22:18,249 workflow DEBUG:
	 Rerunning node:
updatehash = False, self.overwrite = None, self._interface.alway

171109-14:22:18,891 workflow INFO:
	 [Job finished] jobname: quant_cbf_alsop.a00 jobid: 134
171109-14:22:18,893 workflow INFO:
	 Executing: linearReg.a00 ID: 137
171109-14:22:18,907 workflow INFO:
	 [Job finished] jobname: linearReg.a00 jobid: 137
171109-14:22:18,912 workflow INFO:
	 Executing: calc_bright_thresh.a00 ID: 135
171109-14:22:18,943 workflow INFO:
	 Executing node calc_bright_thresh.a00 in dir: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_012/calc_bright_thresh
171109-14:22:18,950 workflow DEBUG:
	 Output dir: ['_report', '_inputs.pklz', '_node.pklz', 'result_calc_bright_thresh.pklz']
171109-14:22:18,956 workflow DEBUG:
	 Output dir: ['_report', '_inputs.pklz', '_node.pklz', 'result_calc_bright_thresh.pklz']
171109-14:22:18,962 workflow DEBUG:
	 Found hashfiles: []
171109-14:22:18,965 workflow DEBUG:
	 Final hashfile: /Volumes/active-19/BABIES/BABIES_perfusion/Workflows/cbfprocflow/_subjid_012/calc_bright_thresh/_0xe9b7767c5fea7319d6c7ee4474a5294

171109-14:22:19,190 workflow ERROR:
	 could not run node: cbfprocflow.calc_bright_thresh.a05
171109-14:22:19,190 workflow INFO:
	 crashfile: /Users/axon/Documents/GitHub/infant_pcasl/crash-20171109-142216-moodlab-calc_bright_thresh.a05-e6254d9f-7061-4a75-8f7d-8baa9782a975.pklz
171109-14:22:19,191 workflow ERROR:
	 could not run node: cbfprocflow.calc_bright_thresh.a04
171109-14:22:19,192 workflow INFO:
	 crashfile: /Users/axon/Documents/GitHub/infant_pcasl/crash-20171109-142217-moodlab-calc_bright_thresh.a04-b4557ab8-0996-48bb-b603-1c0cc0d83025.pklz
171109-14:22:19,193 workflow ERROR:
	 could not run node: cbfprocflow.calc_bright_thresh.a03
171109-14:22:19,194 workflow INFO:
	 crashfile: /Users/axon/Documents/GitHub/infant_pcasl/crash-20171109-142217-moodlab-calc_bright_thresh.a03-fac701e2-149e-445b-8f21-16dcd83bf945.pklz
171109-14:22:19,196 workflow ERROR:
	 could not run node: cbfprocflow.calc_bright_thresh.a02
171109-14:22:19,197 workflow INFO:
	 crashfile: /Users/axon/Documents/Git

RuntimeError: Workflow did not execute cleanly. Check log for details